In [1]:
import pyspark
import os
import shutil
root_folder = "/home/trungdc/unimelb/MAST30024/asm/project 1/"
data_dir = os.path.join(root_folder, "Data/Trip")


import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession
import warnings

# Download yellow trip record to folder: Data/yellow

In [14]:
for file in sorted(os.listdir(os.path.join(data_dir, "yellow"))):
    warnings.filterwarnings("ignore")
    spark = SparkSession.builder.getOrCreate()
    spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
    sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"yellow",file))
    size_before = sdf.count()
    while True:
        sample = sdf.sample(withReplacement=False, fraction=0.3)
        size_after = sample.count()
        if 0.2 < size_after/size_before <= 0.3:
            print(f"File {file} is sampled with size {sample.count()/size_before} or {size_after} rows ")
            sample.repartition(1).write.csv(os.path.join(data_dir,"sample", file), header=True)
            for outfile in os.listdir(os.path.join(data_dir, "sample", file)):
                if outfile.endswith(".csv"):
                    os.rename(os.path.join(data_dir, "sample", file,outfile), os.path.join(data_dir, "after_sample", file ))
            shutil.rmtree(os.path.join(data_dir, "sample" ,file ))
            os.remove(os.path.join(data_dir, "yellow" ,file ))
            break

File yellow_tripdata_2018-01.csv is sampled with size 0.2999213230692588 or 2627273 rows 
File yellow_tripdata_2018-02.csv is sampled with size 0.2997702799645222 or 2545672 rows 
File yellow_tripdata_2018-03.csv is sampled with size 0.2999833728792998 or 2828956 rows 
File yellow_tripdata_2018-04.csv is sampled with size 0.29981102604208365 or 2789896 rows 
File yellow_tripdata_2018-05.csv is sampled with size 0.2998641704853924 or 2765966 rows 
File yellow_tripdata_2018-06.csv is sampled with size 0.29986546675050274 or 2612977 rows 
File yellow_tripdata_2018-07.csv is sampled with size 0.29974006808881 or 2352884 rows 
File yellow_tripdata_2018-08.csv is sampled with size 0.2999840491957457 or 2354615 rows 
File yellow_tripdata_2018-09.csv is sampled with size 0.2998742682490451 or 2411029 rows 
File yellow_tripdata_2018-10.csv is sampled with size 0.2999733026644621 or 2646096 rows 
File yellow_tripdata_2018-11.csv is sampled with size 0.2997749339362596 or 2441716 rows 
File yello

# Concaternate sampled data

In [15]:
warnings.filterwarnings("ignore")
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"after_sample","*.csv"))
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sdf.createOrReplaceTempView("trip")
sdf.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.70,1,N,239,140,2,14,0.5,0.5,0,0,0.3,15.3
1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.20,1,N,140,257,2,33.5,0.5,0.5,0,0,0.3,34.8
1,2018-01-01 00:56:38,2018-01-01 01:01:05,1,1.00,1,N,238,24,1,5.5,0.5,0.5,1.7,0,0.3,8.5
1,2018-01-01 00:52:54,2018-01-01 01:17:33,1,3.50,1,N,141,113,2,16.5,0.5,0.5,0,0,0.3,17.8
1,2018-01-01 00:31:11,2018-01-01 01:07:56,1,10.90,1,N,229,61,2,35,0.5,0.5,0,0,0.3,36.3


In [16]:
print(f"Size of data after being sample: {sdf.count(), len(sdf.columns)}")

Size of data after being sample: (56135731, 17)


In [ ]:
sql_query = """
SELECT *
FROM trip
ORDER BY "tpep_pickup_datetime"
"""
file = "yellow_concat.csv"
out = spark.sql(sql_query)
out.head()
out.repartition(1).write.csv(os.path.join(data_dir, file), header=True)

# Note
- Go to this directory: root_folder/Trip/Data/yellow_concat.csv/
- Inside there will be a single csv file with strange name
- Move the file to this directory root_folder/Trip/Data/
- Delete the folder: root_folder/Trip/Data/yellow_concat.csv/
- Rename that strange-name csv file to yellow_concat.csv

Sorry for this inconvenience. I try to automate but run into a bug that I can not fix